In [1]:
import os
def listdir_nohidden(path):
    for f in os.listdir(path):
        if not f.startswith('.'):
            yield f
# data preprocessing
data_pth = '/Users/yifanli/Desktop/P2/mytest/'
tumor_path_all =data_pth+'tumor_data'
tumor_patients = np.load(data_pth+'data_utils/tumor_patients.npy')
patients=np.load(data_pth+'data_utils/H_patients.npy')


images_path = data_pth+'H_data' + '/' + 'Original'
labels_path = data_pth+ 'H_data' + '/' + 'STAPLE'


skull_brain_path = []
brain_masks = []

for p_fold in listdir_nohidden(images_path):
    patient_id = p_fold[:-7]
    if len(patient_id)>2 and 'nii.gz' in p_fold:
        BrainSkull_path = images_path + '/' + patient_id + '.nii.gz'
        Brain_mask = labels_path + '/' + patient_id + '_staple.nii.gz'
        skull_brain_path.append(BrainSkull_path) 
        brain_masks.append(Brain_mask) 

                
skull_brain_path_order = []
for j in range(len(patients)):
    for i in range(len(skull_brain_path)):

        if patients[j] in skull_brain_path[i].split('/')[-1].split('.')[0]:
            skull_brain_path_order.append(skull_brain_path[i])
brain_masks_order = []
for j in range(len(patients)):
    for i in range(len(brain_masks)):

        if  patients[j] in brain_masks[i].split('/')[-1].split('.')[0]:   
     
            brain_masks_order.append(brain_masks[i])     
            
skull_brain_path = skull_brain_path_order
brain_masks = brain_masks_order

tumor_skull_path = []
tumor_brain_masks = []

for data_folder in listdir_nohidden(tumor_path_all):
    folder_path = tumor_path_all + '/' + data_folder
    
    files =[]
    for p_fold in listdir_nohidden(folder_path):
        files.append(p_fold)    
    tumor_skull_path.append(folder_path + '/' + 'T1C.nii.gz')
    if 'T1C_brain_mask_mc.nii.gz' in files:
        tumor_brain_masks.append(folder_path + '/' + 'T1C_brain_mask_mc.nii.gz')
    elif 'T1C_brain_mask_m.nii.gz' in files:
        tumor_brain_masks.append(folder_path + '/' + 'T1C_brain_mask_m.nii.gz')
    else:
        print('error  ',folder_path)
    
tumor_skull_path_order = []
tumor_brain_masks_order = []

for j in range(len(tumor_patients)):
    for i in range(len(tumor_skull_path)):
        if tumor_patients[j] == tumor_skull_path[i].split('/')[-2]:
            tumor_skull_path_order.append(tumor_skull_path[i])
    for i in range(len(tumor_brain_masks)):
        if tumor_patients[j] == tumor_brain_masks[i].split('/')[-2]:
            tumor_brain_masks_order.append(tumor_brain_masks[i])
tumor_skull_path = tumor_skull_path_order
tumor_brain_masks = tumor_brain_masks_order  
print("Num of Healthy: {}, Tumor: {}" .format(len(skull_brain_path),len(tumor_skull_path)))  


Num of Healthy: 359, Tumor: 125


In [3]:
import mcubes
import trimesh
from datasets.preprocess import process_scan
from datasets.GI_generator_support import process_point_cloud_single_patient
import tqdm

center_359 = np.load('data_utils/center359.npy')
center_H = center_359
center_H = [center_H[i] for i in range(len(center_H))]

In [4]:
import mcubes
import trimesh
# Center of Brain Template 
center = np.array([96.0000, 97.9472, 78.2826])

re_sample=True
if re_sample ==True:
    n_pc = 4096
    n_patch = int(n_pc**0.5)

    GI_H = []
    center_gH = []
    for i,mask in enumerate(tqdm.tqdm(brain_masks)):

        input_v = process_scan(mask,mask=True,output_shape=(192,192,192),center=center_H[i])

        smooth = False
        if smooth == False:
            vertices, triangles = mcubes.marching_cubes(input_v, 0)
        else:
            smoothed_sphere = mcubes.smooth(input_v)

        mcubes.export_obj(vertices, triangles, 'Hcurrent.obj')
        m = trimesh.load_mesh('Hcurrent.obj')
        processed_data_single_patient = process_point_cloud_single_patient(m.vertices,center,n_patch,n_patch,catesian=True)
        if processed_data_single_patient.shape[0]< n_patch*n_patch:
            print(i,processed_data_single_patient.shape[0])
        GI_H.append(processed_data_single_patient)
        
    np.save('data_utils/cGI_H_{}rpt_preC_noc.npy'.format(n_pc),np.array(GI_H))



100%|█████████████████████████████████████████| 359/359 [20:37<00:00,  3.45s/it]
